# 국가별 알콜 섭취량 데이터 분석
- 데이터 읽기
- 데이터 정보 확인
- 컬럼명 한글로 변경
    - 원본 데이터 컬럼 정보
    - country : 국가명
    - beer_servings : 맥주 소비량
    - spirit_servings : 증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)
    - wine_servings : 와인 소비량
    - total_litres_of_pure_alcohol : 순수 알코올
    - continent : 대륙
- 결측치 확인
    - `대륙`에 대한 정보가 부족한 것을 확인할 수 있다.
    - drinks.csv 파일을 열어 원인을 확인한다.
    - 대륙의 종류
        - 아시아, 유럽, 아프리카, 북아메리카, 남아메리카, 오세아니아
        - AS, EU, AF, NA, SA, OC
- 결측치 해결
    - 'NA'를 NaN(결측치)로 읽는 것을 방지
    - pd.read_csv 함수의 na_filter=False 파라메터 : 결측치를 읽어올지 여부 지정
    - 옵션 확인 : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html 
- 통계값 구하기
    - 전세계 맥주, 증류주, 와인, 알코올의 소비에 대한 '평균', '최대값', '최소값' 구하기
- 주류 소비량 비교
    - 와인 소비량이 맥주 + 증류주 소비량보다 큰 나라를 검색해,'대륙'을 기준으로 정렬
    - 맥주 소비량이 230 초과이면서, 와인 소비량이 230 초과인 나라를 검색
    - 전체 평균보다 많은 알코올을 섭취하는 대륙과 알코올 평균 소비량 검색
    - 맥주를 가장 많이 소비하는 국가 확인
    - 증류주를 가장 많이 소비하는 국가 확인
    - 평균 맥주 소비량이 가장 많은 대륙 확인
- 데이터 프레임 조작
    - 국가별 주류 소비량 합계를 구해 새로운 컬럼 ('주류소비량') 추가
    - 주류 소비량 대비 알코올 소비량 비율을 구해 새로운 컬럼 ('알코올비율') 추가
        - 알코올비율 = '알코올' / '주류소비량'
        - 알코올비율'이 NaN인 것에 대해 0으로 채우기
    - 알코올비율이 높은 국가(술을 독하게 마시는 국가) TOP5의 ['국가', '주류소비량', '알코올비율'] 정보 조회
    - DataFrame의 index를 ['대륙', '국가'] 로 지정하고, 대륙별, 국가명으로  정렬하여 df로 저장
    - 전세계 평균 소비량과 한국의 소비량을 비교하는 DataFrame 만들기
        <pre>
               맥주	    증류주	     와인	      알코올	  주류소비량	알코올비율
     국가						
South Korea	140.000000	16.000000	9.000000	9.800000	165.000000	0.059394
World Wide	106.160622	80.994819	49.450777	4.717098	236.606218	0.026115
</pre>
    - pivot_table을 사용하여 대륙별(index), '맥주'와 '와인'의 mean, median, max 값을 구합

In [202]:
# 데이터 읽기

import pandas as pd
import numpy as np

drinks = pd.read_csv("drinks.csv", na_filter=False)
# na_filter (결측치 확인)
drinks

drinks.info()
drinks.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       193 non-null    object 
 1   beer_servings                 193 non-null    int64  
 2   spirit_servings               193 non-null    int64  
 3   wine_servings                 193 non-null    int64  
 4   total_litres_of_pure_alcohol  193 non-null    float64
 5   continent                     193 non-null    object 
dtypes: float64(1), int64(3), object(2)
memory usage: 9.2+ KB


,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,AS
1,Albania,89,132,54,4.9,EU
2,Algeria,25,0,14,0.7,AF
3,Andorra,245,138,312,12.4,EU
4,Angola,217,57,45,5.9,AF


In [203]:
# 컬럼명 한글로 변경

drinks.columns=['국가명', '맥주 소비량', '증류주 소비량', '와인 소비량', '순수 알코올', '대륙']

display(drinks)

,국가명,맥주 소비량,증류주 소비량,와인 소비량,순수 알코올,대륙
0,Afghanistan,0,0,0,0.0,AS
1,Albania,89,132,54,4.9,EU
2,Algeria,25,0,14,0.7,AF
3,Andorra,245,138,312,12.4,EU
4,Angola,217,57,45,5.9,AF
...,...,...,...,...,...,...
188,Venezuela,333,100,3,7.7,SA
189,Vietnam,111,2,1,2.0,AS
190,Yemen,6,0,0,0.1,AS
191,Zambia,32,19,4,2.5,AF


In [204]:
# 결측치 확인

drinks.isna().sum()

국가명        0
맥주 소비량     0
증류주 소비량    0
와인 소비량     0
순수 알코올     0
대륙         0
dtype: int64

In [124]:
# 결측치 확인

# drinks['대륙'] = drinks['대륙'].fillna('NA')

In [205]:
# 통계값 구하기
# 평균, 최대, 최소값 구하기

df2 = drinks[['맥주 소비량','증류주 소비량','와인 소비량','순수 알코올']]
df2.agg([np.mean, np.max, np.min])

df2.agg([np.mean, np.max, np.min]).T

,mean,amax,amin
맥주 소비량,106.160622,376.0,0.0
증류주 소비량,80.994819,438.0,0.0
와인 소비량,49.450777,370.0,0.0
순수 알코올,4.717098,14.4,0.0


In [206]:
# 주류 소비량 비교
# 와인 소비량이 맥주 + 증류주 소비량보다 큰 나라를 검색해,'대륙'을 기준으로 정렬

drinks[drinks['와인 소비량'] > (drinks['맥주 소비량'] + drinks['증류주 소비량'])].sort_values('대륙')
# display(drinks[drinks['와인 소비량'] > (drinks['맥주 소비량'] + drinks['증류주 소비량'])].sort_values(ascending=False))

,국가명,맥주 소비량,증류주 소비량,와인 소비량,순수 알코올,대륙
55,Equatorial Guinea,92,0,233,5.8,AF
148,Sao Tome & Principe,56,38,140,4.2,AF
92,Laos,62,0,123,6.2,AS
171,Timor-Leste,1,1,4,0.1,AS
61,France,127,151,370,11.8,EU
83,Italy,85,42,237,6.5,EU
136,Portugal,194,67,339,11.0,EU
6,Argentina,193,25,221,8.3,SA
185,Uruguay,115,35,220,6.6,SA


In [207]:
# 맥주 소비량과 와인 소비량이 230 초과인 나라

drinks[(drinks['맥주 소비량'] > 230) & (drinks['와인 소비량'] > 230)]

,국가명,맥주 소비량,증류주 소비량,와인 소비량,순수 알코올,대륙
3,Andorra,245,138,312,12.4,EU
99,Luxembourg,236,133,271,11.4,EU
156,Slovenia,270,51,276,10.6,EU


In [208]:
# 쌤 버전
# 전체 평균보다 많은 알코올을 섭취하는 대륙과 아코올 평균 소비량 검색

total_mean = drinks['순수 알코올'].mean()
print('알콜 소비량:', total_mean)

inter_mean = drinks.groupby('대륙')['순수 알코올'].mean()
print(inter_mean)

inter_mean[inter_mean > total_mean]

알콜 소비량: 4.717098445595855
대륙
AF    3.007547
AS    2.170455
EU    8.617778
NA    5.995652
OC    3.381250
SA    6.308333
Name: 순수 알코올, dtype: float64


대륙
EU    8.617778
NA    5.995652
SA    6.308333
Name: 순수 알코올, dtype: float64

In [175]:
inter = drinks.groupby('대륙')

inter_avg = inter[['대륙', '순수 알코올']].agg([np.mean])
display(inter_avg)

data_pure = ((drinks['순수 알코올'].mean()) < inter_avg)
display(data_pure)

,순수 알코올
,mean
대륙,
AF,3.007547
AS,2.170455
EU,8.617778
NA,5.995652
OC,3.381250
SA,6.308333


,순수 알코올
,mean
대륙,
AF,False
AS,False
EU,True
NA,True
OC,False
SA,True


In [209]:
# 맥주를 가장 많이 소비하는 국가

drinks.sort_values('맥주 소비량', ascending=False)

,국가명,맥주 소비량,증류주 소비량,와인 소비량,순수 알코올,대륙
117,Namibia,376,3,1,6.8,AF
45,Czech Republic,361,170,134,11.8,EU
62,Gabon,347,98,59,8.9,AF
65,Germany,346,117,175,11.3,EU
98,Lithuania,343,244,56,12.9,EU
...,...,...,...,...,...,...
107,Mauritania,0,0,0,0.0,AF
158,Somalia,0,0,0,0.0,AF
111,Monaco,0,0,0,0.0,EU
128,Pakistan,0,0,0,0.0,AS


In [210]:
# 맥주를 가장 많이 소비하는 국가
# 쌤 버전

drinks[drinks['맥주 소비량'].max() == drinks['맥주 소비량'].values][['국가명', '맥주 소비량']]

,국가명,맥주 소비량
117,Namibia,376


In [99]:
# 증류주 를 가장 많이 소비하는 국가 확인

drinks.sort_values('증류주 소비량', ascending=False)

,국가명,맥주 소비량,증류주 소비량,와인 소비량,순수 알코올,대륙
68,Grenada,199,438,28,11.9,NA
15,Belarus,142,373,42,14.4,EU
73,Haiti,1,326,1,5.9,NA
141,Russian Federation,247,326,73,11.5,AS
144,St. Lucia,171,315,71,10.1,NA
...,...,...,...,...,...,...
158,Somalia,0,0,0,0.0,AF
128,Pakistan,0,0,0,0.0,AS
63,Gambia,8,0,1,2.4,AF
56,Eritrea,18,0,0,0.5,AF


In [211]:
# 증류주 를 가장 많이 소비하는 국가 확인
# 쌤 버전

drinks[drinks['증류주 소비량'].max() == drinks['증류주 소비량'].values][['국가명', '증류주 소비량']]

,국가명,증류주 소비량
68,Grenada,438


In [179]:
# 평균 맥주 소비량이 가장 많은 대륙 확인

beer_avg = drinks.groupby('대륙')['맥주 소비량'].mean()
print(beer_avg)

max_id = np.argmax(beer_avg)
print(max_id)

print(beer_avg.index[max_id], beer_avg[max_id])

대륙
AF     61.471698
AS     37.045455
EU    193.777778
NA    145.434783
OC     89.687500
SA    175.083333
Name: 맥주 소비량, dtype: float64
2
EU 193.77777777777777


In [212]:
# 데이터 프레임 조작
#국가별 주류 소비량 합계를 구해 새로운 컬럼 '주류소비량' 추가

drinks['주류 소비량'] = drinks['맥주 소비량'] + drinks['증류주 소비량'] + drinks['와인 소비량']
drinks

,국가명,맥주 소비량,증류주 소비량,와인 소비량,순수 알코올,대륙,주류 소비량
0,Afghanistan,0,0,0,0.0,AS,0
1,Albania,89,132,54,4.9,EU,275
2,Algeria,25,0,14,0.7,AF,39
3,Andorra,245,138,312,12.4,EU,695
4,Angola,217,57,45,5.9,AF,319
...,...,...,...,...,...,...,...
188,Venezuela,333,100,3,7.7,SA,436
189,Vietnam,111,2,1,2.0,AS,114
190,Yemen,6,0,0,0.1,AS,6
191,Zambia,32,19,4,2.5,AF,55


In [216]:
# 주류 소비량 대비 알코올 소비량 비율을 구해 새로운 컬럼 '알코올 비율' 추가

drinks['알코올 비율'] = drinks['순수 알코올'] / drinks['주류 소비량']
drinks

,국가명,맥주 소비량,증류주 소비량,와인 소비량,순수 알코올,대륙,주류 소비량,알코올 비율
0,Afghanistan,0,0,0,0.0,AS,0,NaN
1,Albania,89,132,54,4.9,EU,275,0.017818
2,Algeria,25,0,14,0.7,AF,39,0.017949
3,Andorra,245,138,312,12.4,EU,695,0.017842
4,Angola,217,57,45,5.9,AF,319,0.018495
...,...,...,...,...,...,...,...,...
188,Venezuela,333,100,3,7.7,SA,436,0.017661
189,Vietnam,111,2,1,2.0,AS,114,0.017544
190,Yemen,6,0,0,0.1,AS,6,0.016667
191,Zambia,32,19,4,2.5,AF,55,0.045455


In [217]:
drinks['알코올 비율'] = drinks['알코올 비율'].

SyntaxError: invalid syntax (Temp/ipykernel_992/3014973616.py, line 1)

In [218]:
# 알코올 비율이 높은 국가(술을 독하게 마시는 국가) TOP5의 ['국가', '주류소비량', '알코올비율'] 정보 조회

drinks[['국가명', '주류 소비량', '알코올 비율']].sort_values('알코올 비율', ascending=False).head(5)

,국가명,주류 소비량,알코올 비율
63,Gambia,9,0.266667
153,Sierra Leone,30,0.223333
124,Nigeria,49,0.185714
179,Uganda,54,0.153704
142,Rwanda,45,0.151111


In [219]:
# DataFrame의 index를 ['대륙', '국가'] 로 지정하고, 대륙별, 국가명으로 정렬하여 df로 저장

drinks = drinks.set_index(['대륙', '국가명']).sort_index()
drinks

맥주 소비량  증류주 소비량  와인 소비량  순수 알코올  주류 소비량    알코올 비율
대륙 국가명                                                            
AF Algeria           25        0      14     0.7      39  0.017949
   Angola           217       57      45     5.9     319  0.018495
   Benin             34        4      13     1.1      51  0.021569
   Botswana         173       35      35     5.4     243  0.022222
   Burkina Faso      25        7       7     4.3      39  0.110256
...                 ...      ...     ...     ...     ...       ...
SA Paraguay         213      117      74     7.3     404  0.018069
   Peru             163      160      21     6.1     344  0.017733
   Suriname         128      178       7     5.6     313  0.017891
   Uruguay          115       35     220     6.6     370  0.017838
   Venezuela        333      100       3     7.7     436  0.017661

[193 rows x 6 columns]

In [220]:
# 전세계 평균 소비량과 한국의 소비량을 비교하는 DataFrame 만들기

drinks = drinks.reset_index() # 인덱스를 다시 컬럼으로 원위치
korea = drinks[drinks['국가명'] == 'South Korea']
korea

,대륙,국가명,맥주 소비량,증류주 소비량,와인 소비량,순수 알코올,주류 소비량,알코올 비율
85,AS,South Korea,140,16,9,9.8,165,0.059394


In [221]:
korea.drop('대륙', axis=1, inplace=True)
korea

C:\Users\user\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,국가명,맥주 소비량,증류주 소비량,와인 소비량,순수 알코올,주류 소비량,알코올 비율
85,South Korea,140,16,9,9.8,165,0.059394


In [222]:
worldwide = pd.DataFrame(drinks.mean())
worldwide

C:\Users\user\AppData\Local\Temp/ipykernel_992/4074995137.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  worldwide = pd.DataFrame(drinks.mean())


,0
맥주 소비량,106.160622
증류주 소비량,80.994819
와인 소비량,49.450777
순수 알코올,4.717098
주류 소비량,236.606218
알코올 비율,0.028001


In [223]:
worldwide = worldwide.T
worldwide

,맥주 소비량,증류주 소비량,와인 소비량,순수 알코올,주류 소비량,알코올 비율
0,106.160622,80.994819,49.450777,4.717098,236.606218,0.028001


In [224]:
# 일반적인 방법으로 컬럼 삽입 (맨 뒤에 위치)

worldwide['국가'] = ['WorldWide']
worldwide

,맥주 소비량,증류주 소비량,와인 소비량,순수 알코올,주류 소비량,알코올 비율,국가
0,106.160622,80.994819,49.450777,4.717098,236.606218,0.028001,WorldWide


In [225]:
worldwide.drop('국가', axis=1, inplace=True)
worldwide

,맥주 소비량,증류주 소비량,와인 소비량,순수 알코올,주류 소비량,알코올 비율
0,106.160622,80.994819,49.450777,4.717098,236.606218,0.028001


In [228]:
# DataFrame의 임의 위치에 컬럼을 삽입
# df.insert(위치, 컬럼, 값)

worldwide.insert(0, '국가명', 'WorldWide')
worldwide

,국가명,맥주 소비량,증류주 소비량,와인 소비량,순수 알코올,주류 소비량,알코올 비율
0,WorldWide,106.160622,80.994819,49.450777,4.717098,236.606218,0.028001


In [229]:
compare = korea.append(worldwide)
compare

,국가명,맥주 소비량,증류주 소비량,와인 소비량,순수 알코올,주류 소비량,알코올 비율
85,South Korea,140.000000,16.000000,9.000000,9.800000,165.000000,0.059394
0,WorldWide,106.160622,80.994819,49.450777,4.717098,236.606218,0.028001


In [230]:
compare = compare.set_index('국가명')
compare

,맥주 소비량,증류주 소비량,와인 소비량,순수 알코올,주류 소비량,알코올 비율
국가명,,,,,,
South Korea,140.000000,16.000000,9.000000,9.800000,165.000000,0.059394
WorldWide,106.160622,80.994819,49.450777,4.717098,236.606218,0.028001


In [232]:
# pivot_table을 사용하여 대륙별(index), '맥주'와 '와인'의 mean, median, max 값을 구해

drinks.pivot_table(index = '대륙', values = ['맥주 소비량', '와인 소비량'], aggfunc=['mean', 'median', 'max'])

mean             median           max       
        맥주 소비량      와인 소비량 맥주 소비량 와인 소비량 맥주 소비량 와인 소비량
대륙                                                    
AF   61.471698   16.264151   32.0    2.0    376    233
AS   37.045455    9.068182   17.5    1.0    247    123
EU  193.777778  142.222222  219.0  128.0    361    370
NA  145.434783   24.521739  143.0   11.0    285    100
OC   89.687500   35.625000   52.5    8.5    306    212
SA  175.083333   62.416667  162.5   12.0    333    221